In [1]:
import numpy as np
import random
import pandas as pd

In [2]:
df = pd.read_csv("./JustRAIGS_Train_labels.csv")
df.head()

,Eye ID,Final Label,Fellow Eye ID,Age,Label G1,Label G2,Label G3,G1 ANRS,G1 ANRI,G1 RNFLDS,...,G3 ANRS,G3 ANRI,G3 RNFLDS,G3 RNFLDI,G3 BCLVS,G3 BCLVI,G3 NVT,G3 DH,G3 LD,G3 LC
0,TRAIN000034,RG,NaN,32.0,RG,RG,NaN,1.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TRAIN000060,RG,NaN,74.0,RG,RG,NaN,1.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TRAIN000068,RG,TRAIN012581,64.0,RG,NRG,RG,1.0,1.0,0.0,...,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
3,TRAIN000100,RG,NaN,77.0,RG,NRG,RG,1.0,1.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
4,TRAIN000106,RG,TRAIN014144,81.0,RG,RG,NaN,1.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
def final_label(row):
    """
    Determine the final labels based on the availability and agreement of grader labels.
    """
    labels = ['ANRS', 'ANRI', 'RNFLDS', 'RNFLDI', 'BCLVS', 'BCLVI', 'NVT', 'DH', 'LD', 'LC']
    final_labels = {}

    if pd.isna(row['Label G3']):
        # Case 1: G3 is NaN, average G1 and G2
        for label in labels:
            final_labels[f'Final {label}'] = (row[f'G1 {label}'] + row[f'G2 {label}']) / 2
    elif pd.isna(row['Label G1']) and pd.isna(row['Label G2']):
        # Case 2: Both G1 and G2 are NaN, take G3 values
        for label in labels:
            final_labels[f'Final {label}'] = row[f'G3 {label}']
    else:
        # Case 3: G3 is not NaN, and at least one of G1 or G2 is not NaN
        if row['Label G3'] == row['Label G1']:
            # G3 agrees with G1
            for label in labels:
                final_labels[f'Final {label}'] = (row[f'G1 {label}'] + row[f'G3 {label}']) / 2
        elif row['Label G3'] == row['Label G2']:
            # G3 agrees with G2
            for label in labels:
                final_labels[f'Final {label}'] = (row[f'G2 {label}'] + row[f'G3 {label}']) / 2
        else:
            # G3 does not agree with either G1 or G2
            for label in labels:
                final_labels[f'Final {label}'] = row[f'G3 {label}']

    return pd.Series(final_labels)

In [4]:
df[['Final ANRS', 'Final ANRI', 'Final RNFLDS', 'Final RNFLDI', 'Final BCLVS', 'Final BCLVI', 'Final NVT', 'Final DH', 'Final LD', 'Final LC']] = df.apply(final_label, axis=1)

In [5]:
df.to_csv('./final_labels.csv', index = False)